<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Boltzmann-Machines" data-toc-modified-id="Boltzmann-Machines-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Boltzmann Machines</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Look-at-data" data-toc-modified-id="Look-at-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Look at data</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Load-train-and-test-data" data-toc-modified-id="Load-train-and-test-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load train and test data</a></span></li><li><span><a href="#Number-of-users-and-movies" data-toc-modified-id="Number-of-users-and-movies-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Number of users and movies</a></span></li><li><span><a href="#convert-data-to-array" data-toc-modified-id="convert-data-to-array-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>convert data to array</a></span></li><li><span><a href="#Convert-data-to-torch-tensor" data-toc-modified-id="Convert-data-to-torch-tensor-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Convert data to torch tensor</a></span></li><li><span><a href="#Convert-ratings-to-0-1-and--1" data-toc-modified-id="Convert-ratings-to-0-1-and--1-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Convert ratings to 0 1 and -1</a></span></li><li><span><a href="#Class-for-RBM" data-toc-modified-id="Class-for-RBM-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Class for RBM</a></span></li><li><span><a href="#instantiate-the-class" data-toc-modified-id="instantiate-the-class-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>instantiate the class</a></span></li><li><span><a href="#Train-the-RBM" data-toc-modified-id="Train-the-RBM-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Train the RBM</a></span></li><li><span><a href="#Test-the-RBM" data-toc-modified-id="Test-the-RBM-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Test the RBM</a></span></li></ul></div>

# Boltzmann Machines

# Imports

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Look at data

In [2]:
%%bash
ls ml-1m/

README
Train_Test_Set_Creation.R
movies.dat
ratings.csv
ratings.dat
test_set.csv
training_set.csv
users.dat


In [3]:
%%bash
head -n 2 ml-1m/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy


# Load data

In [4]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None,
                     engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None,
                    engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None,
                      engine = 'python', encoding = 'latin-1')

movies.head(), users.head(), ratings.head()

(   0                                   1                             2
 0  1                    Toy Story (1995)   Animation|Children's|Comedy
 1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
 2  3             Grumpier Old Men (1995)                Comedy|Romance
 3  4            Waiting to Exhale (1995)                  Comedy|Drama
 4  5  Father of the Bride Part II (1995)                        Comedy,
    0  1   2   3      4
 0  1  F   1  10  48067
 1  2  M  56  16  70072
 2  3  M  25  15  55117
 3  4  M  45   7  02460
 4  5  M  25  20  55455,
    0     1  2          3
 0  1  1193  5  978300760
 1  1   661  3  978302109
 2  1   914  3  978301968
 3  1  3408  4  978300275
 4  1  2355  5  978824291)

In [5]:
%%bash
head -n 3 ml-100k/u1.base
echo
echo
head -n 3 ml-100k/u1.test

1	1	5	874965758
1	2	3	876893171
1	3	4	878542960


1	6	5	887431973
1	10	3	875693118
1	12	5	878542960


In [6]:
%%bash
head -n 2 ml-100k/u1.base | cat -tev

# ^I  means tab    -t gives tab
# $ means line break  -e gives end
# -v gives other non-printable chars

1^I1^I5^I874965758$
1^I2^I3^I876893171$


# Load train and test data

In [7]:
# Preparing the training set and the test set
# load data using pandas
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t',header=None)
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t',header=None)
test_set = np.array(test_set, dtype = 'int')

In [8]:
# load data using numpy
# if data does not have nans we can use np.loadtxt
# which is much faster
# otherwise, use np.genfromtxt

# if we dont know data has nans, we should use genfromtxt.
tr = np.genfromtxt('ml-100k/u1.base', delimiter = '\t',dtype=int)
ts = np.genfromtxt('ml-100k/u1.test', delimiter = '\t',dtype=int)


# here we know there are no nans, so we can use loadtxt which is faster.
# tr = np.loadtxt('ml-100k/u1.base', delimiter = '\t',dtype=int)
# ts = np.loadtxt('ml-100k/u1.test', delimiter = '\t',dtype=int)

np.array_equal(training_set, tr), np.array_equal(test_set, ts)

(True, True)

# Number of users and movies

In [9]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# convert data to array

In [10]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [11]:
pd.DataFrame(training_set).head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Convert data to torch tensor

In [12]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Convert ratings to 0 1 and -1

In [13]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Class for RBM

In [14]:
# Creating the architecture of the Neural Network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

# instantiate the class

In [15]:
# instantiate the class
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# Train the RBM

In [16]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0] # keep -1 rating same, only update rated movies
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3449)
epoch: 2 loss: tensor(0.2499)
epoch: 3 loss: tensor(0.2445)
epoch: 4 loss: tensor(0.2526)
epoch: 5 loss: tensor(0.2462)
epoch: 6 loss: tensor(0.2516)
epoch: 7 loss: tensor(0.2449)
epoch: 8 loss: tensor(0.2485)
epoch: 9 loss: tensor(0.2470)
epoch: 10 loss: tensor(0.2495)


# Test the RBM

In [17]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        # vt>0 are the movies that are rated.
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2475)


In [18]:
# loss = 0.25 means 75 % times recommended correctly.